In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import optuna
import pickle
import plotly.express as px

from collections import Counter
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.inspection import permutation_importance
from lightgbm import LGBMRegressor

import sys
import os

RAND = 10
N_FOLDS = 3
TIMEOUT = 10800
N_TRIALS = 3000

backend_path = os.path.abspath('../backend')
sys.path.append(backend_path)
model_path = os.path.abspath('../models')
sys.path.append(model_path)
from get_metrics import get_metrics_regression
from check_overfitting import check_overfitting

# Import data

In [ ]:
# Чтение DataFrame df в файл data/df.csv
df = pd.read_csv('../data/df.csv')

In [ ]:
df[:3]

In [ ]:
df.info()

In [ ]:
# основные описательные статистики для числовых признаков
df.iloc[:, 1:].describe()

In [ ]:
# основные описательные статистики для булевых и категориальных признаков
df.describe(include=["object", "bool"])

# LabelEncoder

In [ ]:
# создаем LabelEncoder для кодирования категориальных значений
le = LabelEncoder()

# список категориальных столбцов
categorical_cols = df.select_dtypes(include=[object]).columns

# кодируем каждый категориальный столбец
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

In [ ]:
# рассчитываем матрицу корреляции
corr_matrix = df.corr()

# создаем маску
mask = np.abs(corr_matrix) < 0.1

# указывает размер графика
plt.figure(figsize=(18, 10))

# создаем тепловую карту матрицы корреляции
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', square=True, fmt='.2f', mask=mask, linewidths=0.5, linecolor='grey')

# Показываем график
plt.show()

In [ ]:
df.columns

# permutation_importance

In [ ]:
# признаки
X = df[['milliseconds', 'place', 'status', 'tsunami', 'significance', 'data_type', 'country', 'depth',
        'datetime', 'timezone', 'magnitude_bins', 'year', 'month', 'day', 'hour', 'minute', 'second',
        # 'magnitude',
        # 'longitude',
        # 'latitude'
        ]]

# целевые переменные
y = df[[
    'magnitude',
    'longitude',
    'latitude'
    ]]

In [ ]:
def train_permutation_importance(models, X, y, RAND):
    common_features_file = '../data/common_features.pkl'
    if os.path.exists(common_features_file):
        print("Используем предвычисленные общие признаки из", common_features_file)
        with open(common_features_file, 'rb') as f:
            common_features = pickle.load(f)
        return common_features
    else:
        print("Вычисляем важность признаков с помощью permutation importance...")
        important_features_models = []
        for model in models:
            importances_result = permutation_importance(model, X, y, n_repeats=3, random_state=RAND)
            importances = importances_result.importances_mean
            
            # сортируем признаки по важности
            sorted_importances = sorted(zip(X.columns, importances), key=lambda x: x[1], reverse=True)
            
            # выводим важность признаков для каждой модели
            print(f"Модель {model}:")
            important_features_model = [feature for feature, importance in sorted_importances if importance >= 0.05]
            print(important_features_model)
            print()
            
            important_features_models.append(important_features_model)
        
        # находим общие признаки в каждой модели
        feature_counts = Counter(feature for model_features in important_features_models for feature in model_features)
        common_features = [feature for feature, count in feature_counts.items() if count == len(important_features_models)]
        
        with open(common_features_file, 'wb') as f:
            pickle.dump(common_features, f)
        
        print("Общие важные признаки для моделей:")
        print(common_features)
        return common_features

In [ ]:
# создаем список моделей
models = [
    RandomForestRegressor(random_state=RAND),
    MultiOutputRegressor(GradientBoostingRegressor(random_state=RAND)),
    DecisionTreeRegressor(random_state=RAND),
    MultiOutputRegressor(LGBMRegressor(random_state=RAND, verbosity=-1)),
]

# тренируем модели на всей выборке для определения важных признаков при помощи permutation_importance
for model in models:
    model.fit(X, y)

In [ ]:
# вызываем функцию train_permutation_importance
common_features = train_permutation_importance(models, X, y, RAND)

# разделение данных train_test_split

In [ ]:
# признаки из permutation_importance
X = df[list(common_features)]

# целевые переменные
y = df[['magnitude', 'longitude', 'latitude']]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=RAND)

# Baseline 

## RandomForestRegressor

In [ ]:
# модель RandomForestRegressor
rfr = RandomForestRegressor(random_state=RAND)
# обучаем модель
rfr.fit(X_train, y_train)
#  предсказания на тестовой выборке
y_pred_rfr = rfr.predict(X_test)

In [ ]:
# проверка на переобучение модели RandomForestRegressor
check_overfitting(rfr, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели RandomForestRegressor обученной на train
metrics = get_metrics_regression(y_test,
                                 y_pred = y_pred_rfr,
                                 X_test = X_test,
                                 name='RandomForestRegressor_Baseline')
metrics

## DecisionTreeRegressor

In [ ]:
# модель DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state=RAND)
# обучаем модель
dtr.fit(X_train, y_train)
#  предсказания на тестовой выборке
y_pred_dtr = dtr.predict(X_test)

In [ ]:
# проверка на переобучение модели DecisionTreeRegressor
check_overfitting(dtr, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели DecisionTreeRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_dtr,
                           X_test = X_test,
                           name='DecisionTreeRegressor_Baseline')])
metrics

## GradientBoostingRegressor

In [ ]:
# модель GradientBoostingRegressor
gbr = MultiOutputRegressor(GradientBoostingRegressor(random_state=RAND))

# обучаем модель
gbr.fit(X_train, y_train)
#  предсказания на тестовой выборке
y_pred_gbr = gbr.predict(X_test)

In [ ]:
# проверка на переобучение модели GradientBoostingRegressor
check_overfitting(gbr, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели GradientBoostingRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_gbr,
                           X_test = X_test,
                           name='GradientBoostingRegressor_Baseline')])
metrics

## LightGBMRegressor

In [ ]:
# модель LightGBMRegressor
lgbmreg = MultiOutputRegressor(LGBMRegressor(random_state=RAND, verbosity=-1))

# обучаем модель
lgbmreg.fit(X_train, y_train)

# предсказания на тестовой выборке
y_pred_lgbmreg = lgbmreg.predict(X_test)

In [ ]:
# проверка на переобучение модели LightGBMRegressor
check_overfitting(lgbmreg, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели GradientBoostingRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_lgbmreg,
                           X_test = X_test,
                           name='LightGBMRegressor_Baseline')])
metrics

# Optuna & KFold подбор гиперпараметров и кросс-валидация

## RandomForestRegressor

In [ ]:
# целевая функция для оптимизации
def objective(trial):
    global best_score
    # гиперпараметры для настройки
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 3)
    ccp_alpha = trial.suggest_float('ccp_alpha', 0.01, 1)
    min_weight_fraction_leaf = trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5)
    max_features = trial.suggest_int('max_features', 1, 10)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    # RandomForestRegressor с гиперпараметрами
    rfr = RandomForestRegressor(n_estimators=n_estimators,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                ccp_alpha = ccp_alpha,
                                min_weight_fraction_leaf=min_weight_fraction_leaf,
                                max_features=max_features,
                                bootstrap=bootstrap,
                                random_state=RAND)

    # выполнение кросс-валидации
    scores = cross_val_score(rfr, X_train, y_train, cv=KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND))
    score = np.mean(scores)

        # добавляем условие для обрезки
    if score > best_score:
        best_score = score

    if score < best_score * 0.99: # обрезаем, если score меньше 99% от лучшего score
        raise optuna.TrialPruned()

    return -score  # Optuna минимизирует целевую функцию, поэтому использую -score

# проверка наличия сохраненной модели
best_params_file = os.path.join(model_path, 'rfr_best_params.pkl')
if os.path.exists(best_params_file):
    print('Модель уже сохранена.')
    with open(best_params_file, 'rb') as f:
        rfr_best_params = pickle.load(f)
    print('Параметры модели:', rfr_best_params.get_params())
else:
    print('Модель не сохранена, выполняем поиск гиперпараметров')
    # выполнение настройки гиперпараметров с Optuna
    study = optuna.create_study(direction='minimize')
    best_score = float('-inf')
    study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT)
    best_params = study.best_params
    # сохранение модели
    rfr_best_params = RandomForestRegressor(**best_params, random_state=RAND)
    with open(best_params_file, 'wb') as f:
        pickle.dump(rfr_best_params, f)
    print('Модель сохранена')
    print('Параметры модели:', rfr_best_params.get_params())

In [ ]:
# обучение модели
rfr_best_params.fit(X_train, y_train)

# предсказания на тестовой выборке
y_pred_rfr_best_params = rfr_best_params.predict(X_test)

In [ ]:
# оценка модели
check_overfitting(rfr_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

# получение метрик
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred=y_pred_rfr_best_params,
                           X_test=X_test,
                           name='RandomForestRegressor_best_params_optuna')])
metrics

## DecisionTreeRegressor

In [ ]:
# целевая функция для оптимизации
def objective(trial):
    global best_score
    # гиперпараметры для настройки
    splitter = trial.suggest_categorical('splitter', ['best', 'random'])
    max_depth = trial.suggest_int('max_depth', 5, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 3)
    ccp_alpha = trial.suggest_float('ccp_alpha', 0.01, 1)
    min_weight_fraction_leaf = trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5)
    max_features = trial.suggest_int('max_features', 1, 10)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 5)

    # DecisionTreeRegressor с гиперпараметрами
    rfr = DecisionTreeRegressor(
                                splitter=splitter,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                ccp_alpha = ccp_alpha,
                                min_weight_fraction_leaf=min_weight_fraction_leaf,
                                max_features=max_features,
                                max_leaf_nodes=max_leaf_nodes,
                                random_state=RAND)

    # выполнение кросс-валидации
    scores = cross_val_score(rfr, X_train, y_train, cv=KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND))
    score = np.mean(scores)

    # добавляем условие для обрезки
    if score > best_score:
        best_score = score

    if score < best_score * 0.99: # обрезаем, если score меньше 99% от лучшего score
        raise optuna.TrialPruned()

    return -score  # Optuna минимизирует целевую функцию, поэтому использую -score

# проверка наличия сохраненной модели
best_params_file = os.path.join(model_path, 'dtr_best_params.pkl')
if os.path.exists(best_params_file):
    print('Модель уже сохранена.')
    with open(best_params_file, 'rb') as f:
        dtr_best_params = pickle.load(f)
    print('Параметры модели:', dtr_best_params.get_params())
else:
    print('Модель не сохранена, выполняем поиск гиперпараметров')
    # выполнение настройки гиперпараметров с Optuna
    study = optuna.create_study(direction='minimize')
    best_score = float('-inf')
    study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT)
    best_params = study.best_params
    # сохранение модели
    dtr_best_params = DecisionTreeRegressor(**best_params, random_state=RAND)
    with open(best_params_file, 'wb') as f:
        pickle.dump(dtr_best_params, f)
    print('Модель сохранена')
    print('Параметры модели:', dtr_best_params.get_params())

In [ ]:
# обучение модели
dtr_best_params.fit(X_train, y_train)

# предсказания на тестовой выборке
y_pred_dtr_best_params = dtr_best_params.predict(X_test)

In [ ]:
# оценка модели
check_overfitting(dtr_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

# получение метрик
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred=y_pred_dtr_best_params,
                           X_test=X_test,
                           name='DecisionTreeRegressor_best_params_optuna')])
metrics

## GradientBoostingRegressor

In [ ]:
# целевая функция для оптимизации
def objective(trial):
    global best_score
    # гиперпараметры для настройки
    learning_rate = trial.suggest_float('estimator__learning_rate', 0.1, 1)
    n_estimators = trial.suggest_int('estimator__n_estimators', 100, 1000)
    subsample = trial.suggest_float('estimator__subsample', 0.1, 1.0)
    min_samples_split = trial.suggest_int('estimator__min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('estimator__min_samples_leaf', 1, 5)
    alpha = trial.suggest_float('alpha', 0.01, 1)
    min_weight_fraction_leaf = trial.suggest_float('estimator__min_weight_fraction_leaf', 0.0, 0.5)
    max_depth = trial.suggest_int('estimator__max_depth', 1, 10)

    # GradientBoostingRegressor с гиперпараметрами
    gbr = MultiOutputRegressor(GradientBoostingRegressor(learning_rate=learning_rate,                                                    
                                                         n_estimators=n_estimators,
                                                         subsample=subsample,
                                                         min_samples_split=min_samples_split,
                                                         min_samples_leaf=min_samples_leaf,
                                                         alpha = alpha,
                                                         min_weight_fraction_leaf=min_weight_fraction_leaf,
                                                         max_depth=max_depth,
                                                         random_state=RAND))

    # выполнение кросс-валидации
    scores = cross_val_score(gbr, X_train, y_train, cv=KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND))
    score = np.mean(scores)

        # добавляем условие для обрезки
    if score > best_score:
        best_score = score

    if score < best_score * 0.99: # обрезаем, если score меньше 99% от лучшего score
        raise optuna.TrialPruned()

    return -score  # Optuna минимизирует целевую функцию, поэтому использую -score

# проверка наличия сохраненной модели
best_params_file = os.path.join(model_path, 'gbr_best_params.pkl')
if os.path.exists(best_params_file):
    print('Модель уже сохранена.')
    with open(best_params_file, 'rb') as f:
        gbr_best_params = pickle.load(f)
    print('Параметры модели:', gbr_best_params.get_params())
else:
    print('Модель не сохранена, выполняем поиск гиперпараметров')
    # выполнение настройки гиперпараметров с Optuna
    study = optuna.create_study(direction='minimize')
    best_score = float('-inf')
    study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT)
    best_params = study.best_params
    # удалить префикс 'estimator__' из ключей
    best_params = {k.replace('estimator__', ''): v for k, v in best_params.items()}
    # сохранение модели
    gbr_best_params = MultiOutputRegressor(GradientBoostingRegressor(**best_params, random_state=RAND))
    with open(best_params_file, 'wb') as f:
        pickle.dump(gbr_best_params, f)
    print('Модель сохранена')
    print('Параметры модели:', gbr_best_params.get_params())

In [ ]:
# обучение модели
gbr_best_params.fit(X_train, y_train)

# предсказания на тестовой выборке
y_pred_gbr_best_params = gbr_best_params.predict(X_test)

In [ ]:
# оценка модели
check_overfitting(gbr_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

# получение метрик
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred=y_pred_gbr_best_params,
                           X_test=X_test,
                           name='GradientBoostingRegressor_best_params_optuna')])
metrics

## LightGBMRegressor

In [ ]:
# целевая функция для оптимизации
def objective(trial):
    global best_score
    # гиперпараметры для настройки
    learning_rate = trial.suggest_float('estimator__learning_rate', 0.01, 1)
    n_estimators = trial.suggest_int('estimator__n_estimators', 100, 1000)
    num_leaves = trial.suggest_int('estimator__num_leaves', 31, 255)
    subsample = trial.suggest_float('estimator__subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('estimator__colsample_bytree', 0.1, 1.0)
    reg_alpha = trial.suggest_float('estimator__reg_alpha', 0.01, 1)
    reg_lambda = trial.suggest_float('estimator__reg_lambda', 0.01, 1)

    # LGBMRegressor с гиперпараметрами
    lgbm = MultiOutputRegressor(LGBMRegressor(learning_rate=learning_rate, 
                                               n_estimators=n_estimators, 
                                               num_leaves=num_leaves, 
                                               subsample=subsample, 
                                               colsample_bytree=colsample_bytree, 
                                               reg_alpha=reg_alpha, 
                                               reg_lambda=reg_lambda, 
                                               random_state=RAND))

    # выполнение кросс-валидации
    scores = cross_val_score(lgbmreg, X_train, y_train, cv=KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND))
    score = np.mean(scores)

    # добавляем условие для обрезки
    if score > best_score:
        best_score = score

    if score < best_score * 0.99: # обрезаем, если score меньше 99% от лучшего score
        raise optuna.TrialPruned()

    return -score  # Optuna минимизирует целевую функцию, поэтому использую -score

# проверка наличия сохраненной модели
best_params_file = os.path.join(model_path, 'lgbmreg_best_params.pkl')
if os.path.exists(best_params_file):
    print('Модель уже сохранена.')
    with open(best_params_file, 'rb') as f:
        lgbmreg_best_params = pickle.load(f)
    print('Параметры модели:', lgbmreg_best_params.get_params())
else:
    print('Модель не сохранена, выполняем поиск гиперпараметров')
    # выполнение настройки гиперпараметров с Optuna
    study = optuna.create_study(direction='minimize')
    best_score = float('-inf')
    study.optimize(objective, n_trials=N_TRIALS, timeout=TIMEOUT)
    best_params = study.best_params
    # удалить префикс 'estimator__' из ключей
    best_params = {k.replace('estimator__', ''): v for k, v in best_params.items()}
    # сохранение модели
    lgbmreg_best_params = MultiOutputRegressor(LGBMRegressor(**best_params, random_state=RAND))
    with open(best_params_file, 'wb') as f:
        pickle.dump(lgbmreg_best_params, f)
    print('Модель сохранена')
    print('Параметры модели:', lgbmreg_best_params.get_params())

In [ ]:
# обучение модели
lgbmreg_best_params.fit(X_train, y_train)

# предсказания на тестовой выборке
y_pred_lgbmreg_best_params = lgbmreg_best_params.predict(X_test)

In [ ]:
# оценка модели
check_overfitting(lgbmreg_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

# получение метрик
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred=y_pred_lgbmreg_best_params,
                           X_test=X_test,
                           name='LightGBMRegressor_best_params_optuna')])
metrics

# сохранение metrics в файл data/metrics.csv
metrics.to_csv('../data/metrics.csv', index=False)

# Функция выбора лучшей модели

In [ ]:
def select_best_model(X_test, y_test,
                      rfr_best_params,
                      dtr_best_params,
                      gbr_best_params,
                      lgbmreg_best_params
                      ):
    """
    Выбирает лучшую модель на основе средней квадратичной ошибки (MSE) на тестовой выборке.

    Параметры:
    - X_test: Тестовые признаки
    - y_test: Тестовая целевая переменная
    - rfr_best_params: Обученная модель случайного леса регрессии
    - dtr_best_params: Обученная модель дерева решений регрессии
    - gbr_best_params: Обученная модель градиентного бустинга регрессии
    - lgbmreg_best_params: Обученная модель LightGBM регрессии

    Возвращает:
    - Лучшую модель на основе MSE
    """
    models = [rfr_best_params,
              dtr_best_params,
              gbr_best_params,
              lgbmreg_best_params,
              ]
    mse_values = []

    for model in models:
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_values.append(mse)

    best_model_index = mse_values.index(min(mse_values))
    best_model = models[best_model_index]

    return best_model

In [ ]:
# используем функцию выбора лучшей модели
best_model = select_best_model(X_test, y_test,
                               rfr_best_params,
                               dtr_best_params,
                               gbr_best_params,
                               lgbmreg_best_params,
                               )
print("Лучшая модель:", best_model)

In [ ]:
# предсказания на тестовой выборке лучшей модели
y_pred_best_model = best_model.predict(X_test)

In [ ]:
df_y_pred_rfr_best_params = pd.DataFrame(y_pred_rfr_best_params, columns=['magnitude', 'longitude', 'latitude'])
df_y_pred_dtr_best_params = pd.DataFrame(y_pred_dtr_best_params, columns=['magnitude', 'longitude', 'latitude'])
df_y_pred_gbr_best_params = pd.DataFrame(y_pred_gbr_best_params, columns=['magnitude', 'longitude', 'latitude'])
df_y_pred_lgbmreg_best_params = pd.DataFrame(y_pred_lgbmreg_best_params, columns=['magnitude', 'longitude', 'latitude'])
df_y_pred_best_model = pd.DataFrame(y_pred_best_model, columns=['magnitude', 'longitude', 'latitude'])

## Сравнение карт моделей и фактических данных

In [ ]:
def create_earthquake_maps(y_test, y_pred_model, model_name):
    """
    Создает 2 карты с магнитудой реальными данными и данными из y_pred_model.
    
    Параметры:
    y_test (DataFrame): Реальные данные.
    y_pred_model (DataFrame): Предсказанные данные.
    model_name (str): Название модели.
    
    Возвращает:
    fig (Figure): "Две карты".
    """
    y_pred_model['model_name'] = model_name
    fig1 = px.scatter_geo(y_test, lat='latitude', lon='longitude', color='magnitude', color_continuous_scale='reds', title='Earthquakes Around the World best vs fact')
    fig2 = px.scatter_geo(y_pred_model, lat='latitude', lon='longitude', color='magnitude', color_continuous_scale='reds', title='Earthquakes Around the World')
    
    for fig in [fig1, fig2]:
        fig.update_layout(geo=dict(landcolor='white', oceancolor='lightblue', showland=True, showocean=True), width=600, height=400)
    
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'geo'}, {'type': 'geo'}]], subplot_titles=['Earthquakes Around the World fact', f'Earthquakes Around the World {model_name}'])
    fig.add_trace(fig1.data[0], row=1, col=1)
    fig.add_trace(fig2.data[0], row=1, col=2)
    fig.update_layout(width=1450, height=600)
    
    return fig

In [ ]:
# Используем функцию отрисовки карты
model_name = best_model.estimator.__class__.__name__
fig = create_earthquake_maps(y_test, df_y_pred_best_model, model_name)
fig.show()

In [ ]:
# Используем функцию отрисовки карты
model_name = rfr_best_params.__class__.__name__
fig = create_earthquake_maps(y_test, df_y_pred_rfr_best_params, model_name)
fig.show()

In [ ]:
# Используем функцию отрисовки карты
model_name = dtr_best_params.__class__.__name__
fig = create_earthquake_maps(y_test, df_y_pred_dtr_best_params, model_name)
fig.show()

In [ ]:
# Используем функцию отрисовки карты
model_name = gbr_best_params.estimator.__class__.__name__
fig = create_earthquake_maps(y_test, df_y_pred_gbr_best_params, model_name)
fig.show()

In [ ]:
# Используем функцию отрисовки карты
model_name = lgbmreg_best_params.estimator.__class__.__name__
fig = create_earthquake_maps(y_test, df_y_pred_lgbmreg_best_params, model_name)
fig.show()

In [ ]:
metrics

In [ ]:
# # Обратное преобразование масштабированных данных в X_train
# X_train[['milliseconds', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second']] = scaler.inverse_transform(X_train[['milliseconds', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second']])

# # Обратное преобразование масштабированных данных в X_test
# X_test[['milliseconds', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second']] = scaler.inverse_transform(X_test[['milliseconds', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second']])

In [ ]:
# декодирование колонки 'country'
#X_train['country'] = le.inverse_transform(X_train['country'])
#X_test['country'] = le.inverse_transform(X_test['country'])